In [19]:
import numpy as np 
import pandas as pd 
from matplotlib import pyplot as plt

#!conda config --env --add channels conda-forge
#!conda install numpy -y
#!conda install pandas -y
#!conda install matplotlib -y